In [64]:
import findspark
import os
import pyspark
import time

from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.sql.functions import col, from_json

In [65]:
findspark.init()

In [66]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell'

In [67]:
spark = SparkSession.builder.appName('MachineMonitor').getOrCreate()

In [68]:
# pyspark.SparkContext(app='MachineMonitor').setLogLevel('ERROR')

In [202]:
sensor_schema = \
    StructType([
        StructField('temperature', StringType(), True),
        StructField('rpm', StringType(), True)
    ])

In [203]:
topic_schema = \
    StructType([ 
        StructField('timestamp', StringType(), True), 
        StructField('id_machine', StringType(), True), 
        StructField('sensors-data', sensor_schema, True)
    ])

In [204]:
df = spark \
    .readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('subscribe', 'sensors-data') \
    .load()

In [205]:
df = df.selectExpr('CAST(value AS STRING)')

In [206]:
# from pyspark.sql.functions import col, from_json
# display(
#   df.select(col('value'), from_json(col('value'), topic_schema, {"mode" : "PERMISSIVE"}))
# )

In [207]:
df = \
    df.withColumn(
        'jsonData', 
        from_json(col('value'), topic_schema)
    ).select('jsonData.*')


In [208]:
df = \
    df.select(
        col('id_machine'), 
        col('sensors-data.temperature').alias('temperature'), 
        col('sensors-data.rpm').alias('rpm')
    )

In [209]:
df.printSchema()

root
 |-- id_machine: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- rpm: string (nullable = true)



---

In [190]:
# df_streaming = df.groupby('id_machine').mean('temperature', 'rpm')

In [210]:
# df_streaming = \
#     df_streaming.select(
#         col('id_machine'),
#         col('avg(temperature)').alias('avg_temperature'),
#         col('avg(rpm)').alias('avg_rpm')
#     )

df_streaming = df

In [211]:
# df_streaming.printSchema()

---

In [212]:
temp_table = f"machine_data_{time.strftime('%H%M%S', time.localtime())}"

In [213]:
# streaming = df_streaming \
#     .writeStream \
#     .queryName(temp_table) \
#     .outputMode('complete') \
#     .format('memory') \
#     .start()

streaming = df \
    .writeStream \
    .queryName(temp_table) \
    .outputMode('append') \
    .format('memory') \
    .start()

23/02/07 16:46:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4ef5cd6c-8b64-4bc6-b1d7-4b060e39930c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/07 16:46:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [214]:
# Streams ativados
# spark.streams.active

In [215]:
# spark.sql(f"SELECT id_machine, avg_temperature, avg_rpm from {temp_table}").show()
# spark.sql(f"SELECT * from {temp_table}").show()

In [216]:
for x in range(10):
    # spark.sql(f"SELECT id_machine, avg_temperature, avg_rpm from {temp_table}").show()
    spark.sql(f"SELECT id_machine, temperature, rpm from {temp_table}").show()
    time.sleep(15)
    
streaming.stop()

+----------+-----------+---+
|id_machine|temperature|rpm|
+----------+-----------+---+
+----------+-----------+---+



+----------+-----------+----+
|id_machine|temperature| rpm|
+----------+-----------+----+
|MACHINE-25|         84|1639|
|MACHINE-10|        132|2485|
|MACHINE-25|         90|1522|
|MACHINE-20|         86|1630|
+----------+-----------+----+



+----------+-----------+----+
|id_machine|temperature| rpm|
+----------+-----------+----+
|MACHINE-25|         84|1639|
|MACHINE-10|        132|2485|
|MACHINE-25|         90|1522|
|MACHINE-20|         86|1630|
|MACHINE-10|        140|1886|
|MACHINE-25|        113|1689|
|MACHINE-15|         54|2390|
+----------+-----------+----+



+----------+-----------+----+
|id_machine|temperature| rpm|
+----------+-----------+----+
|MACHINE-25|         84|1639|
|MACHINE-10|        132|2485|
|MACHINE-25|         90|1522|
|MACHINE-20|         86|1630|
|MACHINE-10|        140|1886|
|MACHINE-25|        113|1689|
|MACHINE-15|         54|2390|
|MACHINE-25|         87|2264|
|MACHINE-20|        131|2272|
+----------+-----------+----+



+----------+-----------+----+
|id_machine|temperature| rpm|
+----------+-----------+----+
|MACHINE-25|         84|1639|
|MACHINE-10|        132|2485|
|MACHINE-25|         90|1522|
|MACHINE-20|         86|1630|
|MACHINE-10|        140|1886|
|MACHINE-25|        113|1689|
|MACHINE-15|         54|2390|
|MACHINE-25|         87|2264|
|MACHINE-20|        131|2272|
|MACHINE-30|         73|1633|
|MACHINE-30|         98|1840|
|MACHINE-20|         50|1965|
|MACHINE-15|        138|2336|
+----------+-----------+----+



+----------+-----------+----+
|id_machine|temperature| rpm|
+----------+-----------+----+
|MACHINE-25|         84|1639|
|MACHINE-10|        132|2485|
|MACHINE-25|         90|1522|
|MACHINE-20|         86|1630|
|MACHINE-10|        140|1886|
|MACHINE-25|        113|1689|
|MACHINE-15|         54|2390|
|MACHINE-25|         87|2264|
|MACHINE-20|        131|2272|
|MACHINE-30|         73|1633|
|MACHINE-30|         98|1840|
|MACHINE-20|         50|1965|
|MACHINE-15|        138|2336|
|MACHINE-30|        135|2324|
|MACHINE-20|         83|1982|
+----------+-----------+----+



+----------+-----------+----+
|id_machine|temperature| rpm|
+----------+-----------+----+
|MACHINE-25|         84|1639|
|MACHINE-10|        132|2485|
|MACHINE-25|         90|1522|
|MACHINE-20|         86|1630|
|MACHINE-10|        140|1886|
|MACHINE-25|        113|1689|
|MACHINE-15|         54|2390|
|MACHINE-25|         87|2264|
|MACHINE-20|        131|2272|
|MACHINE-30|         73|1633|
|MACHINE-30|         98|1840|
|MACHINE-20|         50|1965|
|MACHINE-15|        138|2336|
|MACHINE-30|        135|2324|
|MACHINE-20|         83|1982|
|MACHINE-15|        118|2062|
|MACHINE-15|         50|2062|
+----------+-----------+----+



KeyboardInterrupt: 